# Libraries

In [6]:
from turkishnlp import detector
obj = detector.TurkishNLP()
obj.download()
obj.create_word_set()
import turkishnlp
import json
from time import time
import GetOldTweets3 as got
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
import warnings
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.express as px #visualization
import pandas as pd
from nltk import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
import numpy as np
import re
import string
import preprocessor as p
from snowballstemmer import TurkishStemmer
from textblob import TextBlob


Download is successful


In [49]:
import GetOldTweets3 as got # for old tweets

In [59]:
def get_tweets(query,onceki,sonraki):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query)\
                                            .setSince(onceki)\
                                            .setUntil(sonraki)\
                                            .setMaxTweets(100)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    
    text_tweets = [[tw.text,
                tw.date,
                tw.hashtags] for tw in tweet]
    #df = pd.DataFrame(text_tweets, columns = ['Text', 'Date','Hashtags'])
    return text_tweets

In [149]:
onceki = ['2017-01-01','2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01',
       '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01','2018-01-01','2018-02-01','2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01', '2018-07-01',
       '2018-08-01', '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01','2019-01-01','2019-02-01','2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01', '2019-07-01',
       '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01','2020-02-01', '2020-03-01','2020-04-01']
sonraki = ['2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01',
       '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01','2018-01-01','2018-02-01','2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01', '2018-07-01',
       '2018-08-01', '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01','2019-01-01','2019-02-01','2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01', '2019-07-01',
       '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01','2020-02-01', '2020-03-01','2020-04-01','2020-05-01'] 

In [ ]:
def getData(query):
    b=[]
    for i,j in zip(onceki,sonraki):
        tweet=get_tweets("vitamin",i,j)
        b+=tweet
    df = pd.DataFrame(b, columns = ['Text', 'Date','Hashtags']) 
    df.to_csv(query+".csv")
    return df    

In [83]:
df=getData("vitamin")
df.head()

,Text,Date,Hashtags
0,Moved Last Year? A Tax Deduction May Be Availa...,2017-01-31 19:45:28+00:00,#MHRS #LoHiRealEstate
1,Trick of interview http://fb.me/1Ehjdm0rr,2017-01-31 17:53:25+00:00,
2,14歳少年期の一騎くんかわいすぎて目がくらむ～～～～まぶしい～～～～～～～～～～～～～～～～...,2017-01-31 17:26:25+00:00,
3,Sorununuzun çözümü için yardimmasasi@mhrs.gov....,2017-01-31 17:23:46+00:00,
4,"Biz teşekkür eder, sağlıklı günler dileriz. @G...",2017-01-31 13:00:45+00:00,


In [14]:
stop_word_list = nltk.corpus.stopwords.words('turkish') # stop words
neg=pd.read_csv('negative_words_tr.txt') 
pos=pd.read_csv('positive_words_tr.txt')
def token(values):
    filtered_words = [word for word in values.split() if word not in stop_word_list]
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc
def stemming_tokenizer(text): 
    stemmer = TurkishStemmer()
    return [stemmer.stemWord(w) for w in word_tokenize(text)] # köklere ayırma
def pos_finder(x):
    df_words = set(" ".join(x).split(" "))
    extract_words =  set(neg.eski.to_list()).intersection(df_words)
    return ', '.join(extract_words)
def neg_finder(x):
    df_words = set(" ".join(x).split(" "))
    extract_words =  set(pos.olan.to_list()).intersection(df_words)
    return ', '.join(extract_words)
def fixNeg(neg):
    if not (len(neg)==0):
        return len(neg.split(','))
    else:
        return 0
def fixPos(pos):
    if not (len(pos)==0):
        return len(pos.split(','))
    else:
        return 0

In [153]:
df=df.drop(columns=['Unnamed: 0',"Hashtags"])
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3989 entries, 0 to 3999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    3989 non-null   object
 1   Date    3989 non-null   object
dtypes: object(2)
memory usage: 93.5+ KB


In [155]:
df=df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3791 entries, 0 to 3999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    3791 non-null   object
 1   Date    3791 non-null   object
dtypes: object(2)
memory usage: 88.9+ KB


In [156]:
docs = df['Text']
docs = docs.map(lambda x: re.sub('[,\.!?();:$%&#"]', '', x)) #punc are removed
docs = docs.map(lambda x: x.lower()) # küçük harf
docs = docs.map(lambda x: x.strip())

docs = docs.map(lambda x: token(x))
df['Text'] = docs

In [157]:
df['text_token'] = df['Text'].apply(lambda text: stemming_tokenizer(text))
df.head()

,Text,Date,text_token
0,the vitamin zinc helps a lot,2017-01-31 23:59:52+00:00,"[the, vita, zinc, helps, a, lot]"
1,they say it's one of the hardest logic game ev...,2020-08-01 16:42:33+00:00,"[they, say, it, 's, one, of, the, hardest, log..."
2,it's crazy that i didn't eat today and i'm fin...,2017-01-31 23:59:51+00:00,"[it, 's, crazy, that, i, did, n't, eat, today,..."
3,玄武はほんと刃牙の実写化とかで花山薫くん役回ってきそうだけどチャンべーとか鈴木亮平氏みたいに...,2017-01-31 23:59:41+00:00,[玄武はほんと刃牙の実写化とかで花山薫くん役回ってきそうだけどチャンべーとか鈴木亮平氏みたい...
4,last night i slept early but still i feel v sl...,2017-01-31 23:59:26+00:00,"[last, night, i, slept, early, but, still, i, ..."


In [158]:
df['neg'] = df.text_token.apply(pos_finder)
df['pos'] = df.text_token.apply(neg_finder)

df["len_neg"]=df.neg.map(fixNeg)
df["len_pos"]=df.pos.map(fixPos)

df=df[~((df.len_neg==0) & (df.len_pos==0))] # Burda stratejim eğer iki türlü de negatif ya da pozitif kelime bulamadıysa yabancı kelimedir.

In [159]:
df['Text_count'] = df['Text'].str.split().apply(len).value_counts()
df['polarity'] = df['len_pos'] - df['len_neg']
df['sent_rate']=(df['len_pos'] + df['len_neg'])/ len(df['Text_count'])
df.head()

,Text,Date,text_token,neg,pos,len_neg,len_pos,Text_count,polarity,sent_rate
6,vitaminkapseln mit muschelextrakt mit natürlic...,2017-01-31 23:58:26+00:00,"[vitaminkapseln, mit, muschelextrakt, mit, nat...",mit,,1,0,20.0,-1,0.000961
8,vitamin me ha ha well said definitely for sure,2017-01-31 23:57:22+00:00,"[vita, me, ha, ha, well, saidi, definitely, fo...",,well,0,1,24.0,1,0.000961
15,[5 signs of vitamin d deficiency] is goodhave ...,2017-01-31 23:55:16+00:00,"[[, 5, signs, of, vita, d, deficiency, ], is, ...",is,,1,0,23.0,-1,0.000961
19,ultrasounds are pregnancy needs prenatal vitam...,2017-01-31 23:54:28+00:00,"[ultrasounds, are, pregnancy, needs, prenatal,...",is,,1,0,36.0,-1,0.000961
26,vitamin d is critical for oralhealth since it ...,2017-01-31 23:53:10+00:00,"[vita, d, is, critical, for, oralhealth, si, i...","is, critical",,2,0,18.0,-2,0.001921


In [160]:
df.tail()

,Text,Date,text_token,neg,pos,len_neg,len_pos,Text_count,polarity,sent_rate
3989,i love vitamins yo like i’m really happy they ...,2020-04-30 23:45:19+00:00,"[i, love, vitamins, yo, lik, i, ’, m, really, ...",,love,0,1,NaN,1,0.000961
3990,a good multi vitamin and mineral will help sup...,2020-04-30 23:45:12+00:00,"[a, goodu, multi, vita, and, mineral, will, he...",,support,0,1,NaN,1,0.000961
3992,we know from some of the recent studies on cov...,2020-04-30 23:45:05+00:00,"[we, know, from, some, of, the, recent, studie...",is,ideal,1,1,NaN,0,0.001921
3993,is she okay,2020-04-30 23:45:04+00:00,"[is, she, okay]",is,,1,0,NaN,-1,0.000961
3996,prayers up for sure make sure she getti g a lo...,2020-04-30 23:44:51+00:00,"[prayers, up, for, sure, make, sure, she, get,...",,free,0,1,NaN,1,0.000961


In [161]:
open_pharm = df.drop(columns=["len_neg","len_pos","Text_count","text_token","Text","neg","pos"])

In [162]:
open_pharm.head()

,Date,polarity,sent_rate
6,2017-01-31 23:58:26+00:00,-1,0.000961
8,2017-01-31 23:57:22+00:00,1,0.000961
15,2017-01-31 23:55:16+00:00,-1,0.000961
19,2017-01-31 23:54:28+00:00,-1,0.000961
26,2017-01-31 23:53:10+00:00,-2,0.001921


In [163]:
open_pharm.Date = open_pharm.Date.apply(lambda x: x.split("-")[0]+x.split("-")[1])

In [144]:
open_pharm.head()

,Date,polarity,sent_rate
0,201701,1,0.001227
1,201701,1,0.001227
2,201701,0,0.002454
3,201701,2,0.002454
5,201701,0,0.002454


In [164]:
sentiment_open_pharm = open_pharm.groupby(["Date"]).polarity.mean().reset_index()

In [165]:
sentiment_open_pharm.head()

,Date,polarity
0,201701,-0.615385
1,201702,-0.111111
2,201703,-0.500000
3,201704,-0.333333
4,201705,-0.076923


In [166]:
sentiment_open_pharm.Date.unique()

array(['201701', '201702', '201703', '201704', '201705', '201706',
       '201707', '201708', '201709', '201710', '201711', '201712',
       '201801', '201802', '201803', '201804', '201805', '201806',
       '201807', '201808', '201809', '201810', '201811', '201812',
       '201901', '201902', '201903', '201904', '201905', '201906',
       '201907', '201908', '201909', '201910', '201911', '201912',
       '202001', '202002', '202003', '202004', '202006', '202008'],
      dtype=object)

In [167]:
sentiment_open_pharm = sentiment_open_pharm[sentiment_open_pharm.Date<="202004"]

In [126]:
sentiment_open_pharm = sentiment_open_pharm.rename(columns={"Date":"Period","polarity":"Polarity"})

In [168]:
sentiment_open_pharm.to_csv("vitamin_sentiment_results.csv")

# Recources
GetOldTweet : https://medium.com/@AIY/getoldtweets3-830ebb8b2dab

Turkish-stemmer kütüphanesi türkçe kökler için : https://github.com/otuncelli/turkish-stemmer-python

Pozitif-Negatif Kelimeler [Kaynak 3] : https://www.kaggle.com/rtatman/sentiment-lexicons-for-81-languages

Türkçe NLP :https://pypi.org/project/turkishnlp/ 